# YamboWorkflow: BSE@GW

In this notebook we will compute the BSE@G0W0 absorption spectrum and excitonic eigenvalues by means of the `YamboWorkflow` workchain of the plugin.

It is possible to provide the instruction on how to automatically choose the bands to be included in the BSE Hamiltonian. 
Moreover, the workflow will perform the analysis of the QP eigenvalues in order to find the true band gap and compute the Q-related excitons. 

In [1]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder
from aiida.engine import submit

from aiida_quantumespresso.common.types import ElectronicType

import yaml

qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'Silicon/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)
    
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}

YamboWorkflow = WorkflowFactory('yambo.yambo.yambowf')

builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = codes['pwcode_id'],
            preprocessing_code = codes['yamboprecode_id'],
            code = codes['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides={
                'yres': {"yambo": {
                    "parameters": {
                        "variables": {
                            "NGsBlkXs": [4, "Ry"],
                            "BSENGBlk": [4, "Ry"],
                            "FFTGvecs": [20, "Ry"],
                        },
                    },
                },},
            },
            pseudo_family= options['pseudo_family'],
            #parent_folder=orm.load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: in that case, smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

builder.scf.pw.metadata.options = resources

builder.nscf.pw.metadata.options = builder.scf.pw.metadata.options
builder.yres.yambo.metadata.options = builder.scf.pw.metadata.options

Summary of the main inputs:
BndsRnXs = 150
NGsBlkXs = 4 Ry
BSENGBlk = 4 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 6]


### Providing the inputs for the G0W0 quasiparticles simulation, to be used in the BSE step.

In the following we provide the instructions on the quasiparticles to be computed at the G0W0, and the parameters for the simulation. 
The parameters for the G0W0 run are stored in under `builder.qp` (NB: `builder.yres` is used for the final BSE calculation!).

In [2]:
QP_subset_dict= {
    'range_QP':4, #eV         , default=nscf_gap_eV*1.2
    'range_spectrum':6, #eV

}

QP_subset_dict.update({
    'split_bands':True, #default
    'extend_QP': True,  #default is False
    'consider_only':[4,5], #we explicitely compute only these bands.
    'T_smearing':1e-2, #default
    'qp_per_subset': 20,
    'parallel_runs':4,
})

builder.QP_subset_dict= orm.Dict(dict=QP_subset_dict)
builder.qp = builder.yres #we provide the same inputs for G0W0 and BSE, namely resources and settings. 

#providing the G0W0 input parameters.
params_gw = {
    'arguments': [
        'dipoles',
        'HF_and_locXC',
        'dipoles',
        'gw0',
        'ppa',],
    'variables': {
        'Chimod': 'hartree',
        'DysSolver': 'n',
        'GTermKind': 'BG',
        'NGsBlkXp': [4, 'Ry'],
        "FFTGvecs": [20, "Ry"],
        'BndsRnXp': [[1, 50], ''],
        'GbndRnge': [[1, 50], ''],
        'QPkrange': [[[1, 1, 8, 9]], ''],}}


params_gw = orm.Dict(dict=params_gw)
builder.qp.yambo.parameters = params_gw

In [3]:
builder.yres.yambo.metadata.options

{'stash': {}, 'max_wallclock_seconds': 3600, 'prepend_text': 'export OMP_NUM_THREADS=1', 'resources': {'num_cores_per_mpiproc': 1, 'num_machines': 1, 'num_mpiprocs_per_machine': 1}}

In [4]:
builder.additional_parsing = orm.List(list=['gap_','G_v','gap_GG','gap_GY','gap_GK','gap_KK','gap_GM','lowest_exciton','brightest_exciton'])

#### Submission

In [5]:
run = None

In [6]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/home/max/.conda/envs/aiida/lib/python3.9/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 95a60d62-19b5-4d99-b8d0-5626fdc4af38 (pk: 799) (aiida.workflows:yambo.yambo.yambowf)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance. All the outputs of YamboRestart and YamboCalculation are inherited

In [7]:
run.is_finished_ok

False

In [16]:
!verdi process report {run.pk}

2024-02-19 15:37:58 [161 | REPORT]: [799|YamboWorkflow|start_workflow]: no previous pw calculation found, we will start from scratch
2024-02-19 15:37:58 [162 | REPORT]: [799|YamboWorkflow|start_workflow]:  workflow initilization step completed.
2024-02-19 15:37:58 [163 | REPORT]: [799|YamboWorkflow|can_continue]: the workflow continues with a scf calculation
2024-02-19 15:37:58 [164 | REPORT]: [799|YamboWorkflow|perform_next]: performing a scf calculation
2024-02-19 15:38:00 [165 | REPORT]:   [800|PwBaseWorkChain|run_process]: launching PwCalculation<805> iteration #1
2024-02-19 15:38:05 [166 | REPORT]:   [800|PwBaseWorkChain|results]: work chain completed after 1 iterations
2024-02-19 15:38:05 [167 | REPORT]:   [800|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
2024-02-19 15:38:05 [168 | REPORT]: [799|YamboWorkflow|can_continue]: the workflow continues with a nscf calculation
2024-02-19 15:38:06 [169 | REPORT]: [799|YamboWorkflow|perform_next]: performing a nscf c

In [17]:
run.outputs.output_ywfl_parameters.get_dict()

{'SOC': False,
 'gap_': 1.0622892435014,
 'homo': -0.24918457502127,
 'lumo': 0.81310466848016,
 'QP_pk': 855,
 'c_max': 5,
 'q_ind': 13,
 'v_min': 4,
 'gap_GG': 3.0685401315153,
 'gap_GW': 1.0623,
 'homo_G': -0.24918457502127,
 'lumo_G': 2.819355556494,
 'gap_dft': 0.66494336575866,
 'nscf_pk': 814,
 'homo_dft': 0.0,
 'lumo_dft': 0.66494336575866,
 'GW_k_c_ind': 13,
 'GW_k_v_ind': 1,
 'gap_GG_dft': 2.5556287944734,
 'homo_G_dft': 0.0,
 'lumo_G_dft': 2.5556287944734,
 'lowest_exciton': 2.9350125789642,
 'brightest_exciton': 34.067180633545,
 'candidate_for_BSE': True,
 'lowest_exciton_index': 1,
 'brightest_exciton_index': 3456}

We can see that, among the other, we have `gap_GW`. This is the GW gap as computed from the quasiparticle corrections computed along the BZ (and used in the BSE). 
So, the workflow analyse the merged ndb.QP and finds the true GW gap (which may be at different k-points with respect to the DFT one) and uses it in case we need to determine its Q-point (the transfer momentum between electron and hole composing our excitons).
(*v_min* (*c_max) represents the lowest(highest) valence (conduction) band used in BSE)

In [18]:
run.outputs.nscf_mapping.get_dict()

{'soc': False,
 'gap_': [[1, 1, 4, 4], [13, 13, 5, 5]],
 'gap_GG': [[1, 1, 4, 4], [1, 1, 5, 5]],
 'homo_k': 1,
 'lumo_k': 13,
 'valence': 4,
 'gap_type': 'indirect',
 'conduction': 5,
 'nscf_gap_eV': 0.665,
 'dft_predicted': 'semiconductor/insulator',
 'number_of_kpoints': 16,
 'magnetic_calculation': False}

## How to start from already computed set of QP

It is possible also to skip the QP calculation, if you have already done it.
```python
builder.yres.yambo.QP_corrections = orm.load_node(<QP_db pk>)

bse_params = builder.yres.yambo.parameters.get_dict()
bse_params['variables']['KfnQPdb'] = "E < ./ndb.QP"

builder.yres.yambo.parameters = orm.Dict(dict=bse_params)

if not 'KfnQPdb' in builder.yres.yambo.parameters.get_dict()['variables'].keys():
    raise KeyError ("Key KfnQPdb not found in yambo parameters, try to copy.deepcopy the `bse_params` dict before to update it.") 

builder.parent_folder = orm.load_node(<previous yambo calc pk>).outputs.remote_folder

# To also reuse the computed outputs (like the em1d fragments)
builder.yres.yambo.settings = update_dict(self.ctx.yambo_inputs.yambo.settings, 'COPY_DBS', True)
```